In [2]:
import os
import pathlib
import time
import datetime

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob
import json
import random
import csv
import concurrent.futures
from multiprocessing import freeze_support

from keras import layers
from keras import models
from keras import mixed_precision

seed = 6969
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

replays_dir = pathlib.Path("replays")

In [3]:
pre_segment_size = 12
post_segment_size = 12
prediction_size = 8
segment_size = pre_segment_size + post_segment_size + prediction_size

note_size = 49

input_shape = (segment_size, note_size)

batch_size = 256

In [4]:

def get_leaderboard_replays():
  leaderboard_ids = np.array(tf.io.gfile.listdir(str(replays_dir)))
  random.shuffle(leaderboard_ids)
  val_leaderboard_ids = leaderboard_ids[:int(len(leaderboard_ids)*0.2)]

  train_data = []
  val_data = []
  for leaderboard_id in leaderboard_ids:
    replay_files = glob.glob(f'{replays_dir}/{leaderboard_id}/*.npy')

    if leaderboard_id in val_leaderboard_ids:
      val_data.append((leaderboard_id, replay_files))
    else:
      train_data.append((leaderboard_id, replay_files))

  return train_data, val_data

def get_replay_notes(replay, njs):
  notes = []
  
  prev_zero_note_time = 0
  prev_one_note_time = 0
  
  for note_info, score, note_time in sorted(replay, key=lambda item: item[2]):
    type = note_info[-2]
    
    # NOTE: 0-100 score range is rare and often happens for tracking problems that are not important here
    # would be good to replace this with acc component only and potentially learn all both acc and swing angles
    # but need different format replay files for that
    # score = max(0, score - 100)
    
    delta_to_zero = note_time - prev_zero_note_time
    delta_to_one = note_time - prev_one_note_time
    
    if type == "0":
      prev_zero_note_time = note_time
      note = preprocess_note(score, delta_to_zero, delta_to_one, note_info, njs)
      notes.append(note)
    if type == "1":
      prev_one_note_time = note_time
      note = preprocess_note(score, delta_to_one, delta_to_zero, note_info, njs)
      notes.append(note)
  
  return notes

def preprocess_note(score, delta, delta_other, note_info, njs):
  # NOTE: timing increases difficulty not linearly and caps out at ~2 seconds
  # no idea if such parameters can be learned by neural networks without adding scaling like I did right here
  delta_long = max(0, 2 - delta)/2
  delta_other_long = max(0, 2 - delta_other)/2
  delta_short = max(0, 0.5 - delta)*2
  delta_other_short = max(0, 0.5 - delta_other)*2

  col_number = int(note_info[1])
  row_number = int(note_info[2])
  direction_number = int(note_info[4])

  color = int(note_info[3])

  row_col = [0] * 4 * 3
  direction = [0] * 10
  row_col2 = [0] * 4 * 3
  direction2 = [0] * 10

  row_col[col_number * 3 + row_number] = 1
  direction[direction_number] = 1

  response = []
  
  if color == 0:
      response.extend(row_col)
      response.extend(direction)
      response.extend(row_col2)
      response.extend(direction2)
      response.extend([
          delta_short,
          delta_long,
      ])
      response.extend([
          delta_other_short,
          delta_other_long,
      ])
  if color == 1:
      response.extend(row_col2)
      response.extend(direction2)
      response.extend(row_col)
      response.extend(direction)
      response.extend([
          delta_other_short,
          delta_other_long,
      ])
      response.extend([
          delta_short,
          delta_long,
      ])
  response.extend([
    njs,
    score
  ])
  
  return response

def create_segments(notes):
  empty_res = ([], [])
  if len(notes) < prediction_size:
      return empty_res

  segments = []
  predictions = []
  for i in range(len(notes)-prediction_size+1):
      if i % prediction_size != 0:
          continue
              
      pre_slice = notes[max(0, i-pre_segment_size):i]
      slice = notes[i:i+prediction_size]
      post_slice = notes[i+prediction_size:i+prediction_size+post_segment_size]

      # NOTE: using relative score can be good to find relative difficulty of the notes more fairly
      # because good players will always get higher acc and worse players will do badly even on easy patterns

      pre_segment = [np.array(note[:-1]) for note in pre_slice]
      if len(pre_segment) < pre_segment_size:
          pre_segment[0:0] = [np.zeros(note_size, dtype=np.float32) for i in range(pre_segment_size - len(pre_segment))]
          
      segment = [np.array(note[:-1]) for note in slice]

      post_segment = [np.array(note[:-1]) for note in post_slice]
      if len(post_segment) < post_segment_size:
          post_segment.extend([np.zeros(note_size, dtype=np.float32) for i in range(post_segment_size - len(post_segment))])


      # fix this pls
      prediction = [note[-1] for note in slice]

      final_segment = []
      final_segment.extend(pre_segment)
      final_segment.extend(segment)
      final_segment.extend(post_segment)
      segments.append(final_segment)
      
      predictions.append(prediction)

  return segments, predictions

def to_id(value):
  if int(value) < 30000:
    return str(30000 + int(value))
  return str(int(value))

def preprocess_leaderboard_replays(leaderboard_replays, print_progress=False):
  asd = {}
  replays = []
  
  for leaderboard_replay in leaderboard_replays:
    arr = np.load(leaderboard_replay)
    sort_indices = np.argsort(arr[:, 2])

    # Sort the entire array column-wise based on these indices
    sorted_arr = arr[sort_indices]
    replays.append(sorted_arr)
  
  for replay in replays:
    if len(asd.values()) == 0:
      for values in replay:
        asd[to_id(values[0]) + str(values[2])] = [to_id(values[0]), [values[1]], values[2], 1]
    else:
      for values in replay:
        key = to_id(values[0]) + str(values[2])
        if key in asd:
          asd[key][1].append(values[1])
          asd[key][2] += values[2]
          asd[key][3] += 1
  
  asd2 = []
  for values in asd.values():
    if values[3] < 5 or len(values[0]) > 5:
      return ([], [])

    acc_list = values[1]
    acc = sum(acc_list) / len(acc_list) if len(acc_list) > 0 else 0
    asd2.append([values[0], acc, values[2] / values[3]])
  
  notes = get_replay_notes(asd2, float(leaderboard_replays[0].split("-")[2].replace(".npy", "")))
  return create_segments(notes)


def generate_data(leaderboards_replays):
  segments = []
  scores = []
  
  for (leaderboard_id, leaderboard_replays) in leaderboards_replays:
    segment, score = preprocess_leaderboard_replays(leaderboard_replays)
    if len(segment) == 0: 
      continue

    segments.extend(segment)
    scores.extend(score)
  
  return np.array(segments), np.array(scores)

def clamp(num, lower, higher):
  return min([max([num, lower]), higher])

train_data, val_data = get_leaderboard_replays()
test_data = val_data

train_x, train_y = generate_data(train_data)
val_x, val_y = generate_data(val_data)

KeyboardInterrupt: 

In [29]:
from keras import Input, Model
from keras.layers import Bidirectional, LSTM, Concatenate, Cropping1D, TimeDistributed, Dense

# Input layer
input_layer = Input(shape=input_shape, name="input_1")

bidirectional_1 = layers.Bidirectional(layers.LSTM(units=12, return_sequences=True, activation="tanh"), backward_layer=layers.LSTM(units=12, return_sequences=True, go_backwards=True, activation="tanh"))(input_layer)
bidirectional_2 = layers.Bidirectional(layers.LSTM(units=10, return_sequences=True, activation="tanh"), backward_layer=layers.LSTM(units=10, return_sequences=True, go_backwards=True, activation="tanh"))(bidirectional_1)

# Concatenate the output of the second bidirectional LSTM layer with the input
concatenate = Concatenate()([bidirectional_2, input_layer])

# Cropping1D layer
cropping1d = Cropping1D(cropping=(12, 12))(concatenate)  # Adjust the cropping as necessary

# TimeDistributed layer with Dense layer inside
time_distributed_1 = TimeDistributed(Dense(units=16, activation='relu'))(cropping1d)

# Another TimeDistributed layer with Dense layer inside
time_distributed_2 = TimeDistributed(Dense(units=16, activation='relu'))(time_distributed_1)

# Final TimeDistributed layer with Dense layer for output
time_distributed_3 = TimeDistributed(Dense(units=1, activation='linear'), name="time_distributed_2")(time_distributed_2)

# Create the model
model = Model(inputs=input_layer, outputs=time_distributed_3)

model.summary()


model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss=tf.keras.losses.MeanAbsoluteError(reduction="sum_over_batch_size"),
    metrics=['mae', 'mse'],
)
  
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_1             │ (None, 32, 49)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_10    │ (None, 32, 24)    │      5,952 │ input_1[0][0]     │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_11    │ (None, 32, 20)    │      2,800 │ bidirectional_10… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 32, 69)    │          0 │ bidirectional_11… │
│ (Concatenate)       │                   │            │ input_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cropping1d_5        │ (None, 8, 69)     │          0 │ concatenate_5[0]… │
│ (Cropping1D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_13 │ (None, 8, 16)     │      1,120 │ cropping1d_5[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_14 │ (None, 8, 16)     │        272 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 8, 1)      │         17 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,161 (39.69 KB)

 Trainable params: 10,161 (39.69 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = model.fit(
    train_x,
    train_y,
    validation_data=(val_x, val_y),
    batch_size=batch_size,
    epochs=20,
    shuffle=True,
    callbacks=[tf.keras.callbacks.EarlyStopping(verbose=1, patience=20), tensorboard_callback]
)

model.save('model_sleep_bl.keras')

Epoch 1/20
895/895 ━━━━━━━━━━━━━━━━━━━━ 35s 29ms/step - loss: 0.0884 - mae: 0.0884 - mse: 0.0164 - val_loss: 0.0670 - val_mae: 0.0672 - val_mse: 0.0091
Epoch 2/20
895/895 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - loss: 0.0667 - mae: 0.0667 - mse: 0.0089 - val_loss: 0.0657 - val_mae: 0.0658 - val_mse: 0.0084
Epoch 3/20
895/895 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - loss: 0.0655 - mae: 0.0655 - mse: 0.0086 - val_loss: 0.0655 - val_mae: 0.0656 - val_mse: 0.0080
Epoch 4/20
895/895 ━━━━━━━━━━━━━━━━━━━━ 24s 26ms/step - loss: 0.0645 - mae: 0.0645 - mse: 0.0084 - val_loss: 0.0649 - val_mae: 0.0650 - val_mse: 0.0078
Epoch 5/20
895/895 ━━━━━━━━━━━━━━━━━━━━ 26s 29ms/step - loss: 0.0636 - mae: 0.0636 - mse: 0.0081 - val_loss: 0.0662 - val_mae: 0.0663 - val_mse: 0.0077
Epoch 6/20
895/895 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - loss: 0.0629 - mae: 0.0629 - mse: 0.0080 - val_loss: 0.0625 - val_mae: 0.0626 - val_mse: 0.0074
Epoch 7/20
895/895 ━━━━━━━━━━━━━━━━━━━━ 29s 33ms/step - loss: 0.0626 - mae: 0.0626 - mse

In [31]:
import tensorflow as tf
import tf2onnx
import onnx

model.save("model_sleep_bl.keras")

# Convert the saved model to ONNX
spec = (tf.TensorSpec((None, segment_size, note_size), tf.float32, name="input_1"),)
output_path = "model_sleep_bl.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=None, output_path=output_path)

print(f"Model is converted to ONNX and saved at {output_path}")

Model is converted to ONNX and saved at model_sleep_bl.onnx


In [32]:
predictions = []
for leaderboard_id, leaderboard_replays in test_data:
    try:
      curr, score = preprocess_leaderboard_replays(leaderboard_replays)

      _predictions = model.predict(np.array(curr))

      real_sum = 0
      for prediction in score:
        real_sum += prediction[0]

      real_avg = real_sum/len(score)
      real_percentage_score = (100 + real_avg * 15) / 115

      prediction_sum = 0
      for prediction in _predictions:
        prediction_sum += prediction[0]

      avg = prediction_sum/len(_predictions)
      percentage_score = (100 + avg[0] * 15) / 115

      predictions.append([f"https://beatleader.xyz/leaderboard/global/{leaderboard_id}", round(percentage_score, 5), round(real_percentage_score, 5), abs(round(real_percentage_score - percentage_score, 5))])
    except KeyboardInterrupt:
      raise
    except Exception as e:
      print(e)
      continue
with open('predictions.csv', 'w', newline='', encoding='utf-8') as f:
      writer = csv.writer(f)
      header = ["LeaderboardId", "Prediction", "Expected", "Difference"]
      writer.writerow(header)

      for prediction in predictions:
        writer.writerow(prediction)


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
math domain error
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
math domain error
6/6 ━━━

: 